<a href="https://colab.research.google.com/github/ghdwjdduq/Sentimental-Analysis-on-Anti-Vaccine-Tweets/blob/main/Demo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers

In [ ]:
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
import tensorflow as tf
import numpy as np

In [ ]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

Mounted at /gdrive


In [ ]:
model_save_path='/gdrive/Shared drives/COMP484-Project/ModellingFiles/models/dbert_model.h5'

# bert_model.save("trained_model")

trained_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased',num_labels=3)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
metric = tf.keras.metrics.SparseCategoricalAccuracy('accuracy')
optimizer = tf.keras.optimizers.Adam(learning_rate=2e-5,epsilon=1e-08)

trained_model.compile(loss=loss,optimizer=optimizer, metrics=[metric])
trained_model.load_weights(model_save_path)

Some layers from the model checkpoint at distilbert-base-uncased were not used when initializing TFDistilBertForSequenceClassification: ['vocab_layer_norm', 'vocab_transform', 'activation_13', 'vocab_projector']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some layers of TFDistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier', 'classifier', 'dropout_39']
You should probably TRAIN this model on a down-stream task to be able to use i

In [ ]:
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

In [ ]:
def sentimental_analysis(text, trained_model):
  input = tokenizer(text, return_tensors='tf')
  prediction = trained_model(**input)
  Encoder = {0: "Government Conspiracy", 1: "Medical Choice", 2: "Risk of Vaccine"}
  pred = prediction[0][0]
  argmax = np.argmax(pred.numpy())
  return Encoder[argmax]

In [ ]:
text = "Bill Gates will rule the world with this vaccine"
print(sentimental_analysis(text, trained_model))

text = "I got the vaccine and I died"
print(sentimental_analysis(text, trained_model))

text = "Vaccine has risk"
print(sentimental_analysis(text, trained_model))

text = "@haveaconcern I’m telling you they want us all genetically modified or dead.😡#NoVaccineForMe"
print(sentimental_analysis(text, trained_model))

text = "@JimDelos147 @cherese88419639 @Aaron_Stevens81 @danboy The world can be made better by vaccine deaths and depopulation. Sure."
print(sentimental_analysis(text, trained_model))

text = "@JimDelos147 @cherese88419639 @Aaron_Stevens81 @danboy The world can be made better by vaccine deaths. Sure."
print(sentimental_analysis(text, trained_model))


Government Conspiracy
Government Conspiracy
Risk of Vaccine
Government Conspiracy
Government Conspiracy
Risk of Vaccine


##Demo

In [ ]:
demo = True
while demo:
  text = input("Write a sentence to analyze the intent: ")
  result = sentimental_analysis(text, trained_model)
  print ("The result is", result)
  repeat = input("\npress q to stop, y to continue: ")
  if repeat == "q":
    demo = False

Write a sentence to analyze the intent: I don’t take medical advice from anybody who has ever publicly advocated human depopulation
The result is Government Conspiracy

press q to stop, y to continue: y
Write a sentence to analyze the intent: What if I die from blood clot?
The result is Risk of Vaccine

press q to stop, y to continue: q


# **Speech to Text using Google API**

In [ ]:
!pip3 install SpeechRecognition

In [ ]:
!pip install ffmpeg-python

In [ ]:
from IPython.display import HTML, Audio
from google.colab.output import eval_js
from base64 import b64decode
import numpy as np
from scipy.io.wavfile import read as wav_read
import io
import ffmpeg

In [ ]:
AUDIO_HTML = """
<script>
var my_div = document.createElement("DIV");
var my_p = document.createElement("P");
var my_btn = document.createElement("BUTTON");
var t = document.createTextNode("Press to start recording");

my_btn.appendChild(t);
//my_p.appendChild(my_btn);
my_div.appendChild(my_btn);
document.body.appendChild(my_div);

var base64data = 0;
var reader;
var recorder, gumStream;
var recordButton = my_btn;

var handleSuccess = function(stream) {
  gumStream = stream;
  var options = {
    //bitsPerSecond: 8000, //chrome seems to ignore, always 48k
    mimeType : 'audio/webm;codecs=opus'
    //mimeType : 'audio/webm;codecs=pcm'
  };            
  //recorder = new MediaRecorder(stream, options);
  recorder = new MediaRecorder(stream);
  recorder.ondataavailable = function(e) {            
    var url = URL.createObjectURL(e.data);
    var preview = document.createElement('audio');
    preview.controls = true;
    preview.src = url;
    document.body.appendChild(preview);

    reader = new FileReader();
    reader.readAsDataURL(e.data); 
    reader.onloadend = function() {
      base64data = reader.result;
      //console.log("Inside FileReader:" + base64data);
    }
  };
  recorder.start();
  };

recordButton.innerText = "Recording... press to stop";

navigator.mediaDevices.getUserMedia({audio: true}).then(handleSuccess);


function toggleRecording() {
  if (recorder && recorder.state == "recording") {
      recorder.stop();
      gumStream.getAudioTracks()[0].stop();
      recordButton.innerText = "Saving the recording... pls wait!"
  }
}

// https://stackoverflow.com/a/951057
function sleep(ms) {
  return new Promise(resolve => setTimeout(resolve, ms));
}

var data = new Promise(resolve=>{
//recordButton.addEventListener("click", toggleRecording);
recordButton.onclick = ()=>{
toggleRecording()

sleep(2000).then(() => {
  // wait 2000ms for the data to be available...
  // ideally this should use something like await...
  //console.log("Inside data:" + base64data)
  resolve(base64data.toString())

});

}
});
      
</script>
"""

def get_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  sr, audio = wav_read(io.BytesIO(riff))
  return audio, sr

def save_audio():
  display(HTML(AUDIO_HTML))
  data = eval_js("data")
  binary = b64decode(data.split(',')[1])
  
  process = (ffmpeg
    .input('pipe:0')
    .output('pipe:1', format='wav')
    .run_async(pipe_stdin=True, pipe_stdout=True, pipe_stderr=True, quiet=True, overwrite_output=True)
  )
  output, err = process.communicate(input=binary)
  
  riff_chunk_size = len(output) - 8
  # Break up the chunk size into four bytes, held in b.
  q = riff_chunk_size
  b = []
  for i in range(4):
      q, r = divmod(q, 256)
      b.append(r)

  # Replace bytes 4:8 in proc.stdout with the actual size of the RIFF chunk.
  riff = output[:4] + bytes(b) + output[8:]

  with open('audio.wav','wb') as f:
    f.write(riff)
  return 'audio.wav'  # or webm ?

In [ ]:
audio = save_audio()

In [ ]:
import speech_recognition as sr
r = sr.Recognizer()
# open the file
def SpeechText(audio):
  with sr.AudioFile(audio) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    return(text)

SpeechText(audio)

"I don't like this"

# **Final Demo**

In [121]:
demo = True
while demo:
  print("Please read your tweets: ")
  audio = save_audio()
  text = SpeechText(audio)
  print("Tweets you read to me is:", text)
  result = sentimental_analysis(text, trained_model)
  print ("The sentimental anlysis of your tweet is", result)
  repeat = input("\npress q to stop, y to continue: ")
  if repeat == "q":
    demo = False

Please read your tweets: 


Tweets you read to me is: I hate your violin
The sentimental anlysis of your tweet is Government Conspiracy

press q to stop, y to continue: q
